In [1]:
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch

train = pd.read_csv("train.csv")
dev = pd.read_csv("dev.csv")
test = pd.read_csv("test.csv")
dev.shape

(10000, 3)

In [2]:
test.head()

,Id,text,Category
0,60ffed24da8262dd38c916dd517c617641710ede348320...,撿到 學妹 的 手機 : 吃完 飯 看到 學妹 疑 落 手機 在 三 顧 茅蘆 的 ...,0
1,cb931d414e634344c782a128d542188d6ec1d325e82dd1...,這些 年 努力 的 付出 ~ 但是 最終 有 誰 會 記得 大家 看到 的 只 是 表面...,0
2,f7febf0abfefcb927ee16440eaca62d795392de2d685a6...,幫 我 把 摩托車 鑰匙 放到 放 飲料 的 地方,0
3,709399d1be2207b8f06dfc9ddb3a0709d9894c02849b88...,在一起 是 需要 2 人 同意 但 分手 卻 是 要 一 人 同意 就 好 了 是 ...,0
4,0d1969c03277dfaef7ccfd40f0710bcff4ecd978faa935...,只好 說 抱歉,0


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

BOW_vectorizer = CountVectorizer() 
BOW_500 = CountVectorizer(max_features=10000) 
BOW_500.fit(train['text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [3]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

tfIdfVectorizer_1000 = TfidfVectorizer(max_features = 10000, tokenizer=nltk.word_tokenize)

tfIdfVectorizer_1000.fit(train['text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function word_tokenize at 0x7fe21dc57e60>, use_idf=True,
        vocabulary=None)

In [7]:
X_train = tfIdfVectorizer_1000.transform(train['text'])
y_train = train['Category']

X_dev = tfIdfVectorizer_1000.transform(dev['text'])
y_dev = dev['Category']

X_test = tfIdfVectorizer_1000.transform(test['text'])
y_test = test['Category']

print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_dev.shape: ', X_train.shape)
print('y_dev.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (80000, 10000)
y_train.shape:  (80000,)
X_dev.shape:  (80000, 10000)
y_dev.shape:  (80000,)
X_test.shape:  (10000, 10000)
y_test.shape:  (10000,)


In [8]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
import tensorflow as tf
from tensorflow import keras

input_shape = 10000
output_shape = 1

# input layer
model_input = Input(shape=(input_shape, ))
X = model_input

# 1st hidden layer
X_W1 = Dense(units=4096)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=2048)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=1024)(H2)  # 4
H3 = ReLU()(H2_W3)

H3_W4 = Dense(units=512)(H3)  # 4
H4 = ReLU()(H3_W4)

H4_W5 = Dense(units=256)(H4)  # 4
H5 = ReLU()(H4_W5)

H5_W6 = Dense(units=128)(H5)  # 4
H6 = ReLU()(H5_W6)

H6_W7 = Dense(units=64)(H6)  # 4
H7 = ReLU()(H6_W7)

H7_W8 = Dense(units=32)(H7)  # 4
H8 = ReLU()(H7_W8)

H8_W9 = Dense(units=16)(H8)  # 4
H9 = ReLU()(H8_W9)

H9_W10 = Dense(units=output_shape)(H9)  # 4
H10 = keras.activations.softmax(H9_W10)

model_output = H10

# create model
model = Model(inputs=[model_input], outputs=[model_output])

opt = keras.optimizers.Adam(learning_rate=0.01)
# loss function & optimizer
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 10000)]           0         
_________________________________________________________________
dense_10 (Dense)             (None, 4096)              40964096  
_________________________________________________________________
re_lu_9 (ReLU)               (None, 4096)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 2048)              8390656   
_________________________________________________________________
re_lu_10 (ReLU)              (None, 2048)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
re_lu_11 (ReLU)              (None, 1024)              0   

In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)


In [73]:
y_dev = label_encode(label_encoder, y_dev)

In [9]:
X_train = X_train.todense()
X_dev = X_dev.todense()
X_test = X_test.todense()
# training setting

epochs = 13
batch_size = 32

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    validation_data = (X_dev, y_dev))
print('training finish')

Epoch 1/13
 468/2500 [====>.........................] - ETA: 7:12 - loss: 7.6521 - accuracy: 0.4982

KeyboardInterrupt: 